# Backtesting a Strategy

We are ready to backtest a simple trading strategy with **PyBroker**! Let's begin by importing the needed classes below:

In [1]:
import pybroker
from pybroker import Strategy, StrategyConfig, YFinance

pybroker.enable_data_source_cache('my_strategy')

[Yahoo Finance](https://finance.yahoo.com) will be used as the [DataSource](https://www.pybroker.com/en/latest/reference/pybroker.data.html#pybroker.data.DataSource) of our backtest. We also enable data source caching so that data is only downloaded one time when running backtests. 

The next step is to instantiate a new [Strategy](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy) instance that will be used to backtest our trading strategy:

In [2]:
config = StrategyConfig(initial_cash=500_000)
strategy = Strategy(YFinance(), '3/1/2017', '3/1/2022', config)

The above creates a ```Strategy``` that will download data between ```3/1/2017``` and ```3/1/2022``` from Yahoo Finance before running a backtest. 

A [StrategyConfig](https://www.pybroker.com/en/latest/reference/pybroker.config.html#pybroker.config.StrategyConfig) is used to configure the ```Strategy```. By default, the [initial_cash](https://www.pybroker.com/en/latest/reference/pybroker.config.html#pybroker.config.StrategyConfig.initial_cash) to use for the backtest would be ```100_000```, but the config overrides the default with a value of ```500_000```. For more configuration options, see the [StrategyConfig reference documentation](https://www.pybroker.com/en/latest/reference/pybroker.config.html#pybroker.config.StrategyConfig).

## Defining Strategy Rules

Let's now implement a basic trading strategy with the following rules:

- Buy shares in a stock if:

    - There is no open long position in that stock.
    
    - The last close price is less than the low of the previous bar.
    
- Set the limit price of the buy order to ```0.01``` less than the last close price.
- Hold the position for 3 days before liquidating it at the current market price.
- Trade the rules on ```AAPL``` and ```MSFT```, allocating up to 25% of the portfolio to each.

In [3]:
def buy_low(ctx):
    # If shares were already purchased and are currently being held, then return.
    if ctx.long_pos():
        return
    # If the latest close price is less than the previous day's low price,
    # then place a buy order.
    if len(ctx.low) >= 2 and ctx.close[-1] < ctx.low[-2]:
        # Buy a number of shares that is equal to 25% the portfolio.
        ctx.buy_shares = ctx.calc_target_shares(0.25)
        # Set the limit price of the order.
        ctx.buy_limit_price = ctx.close[-1] - 0.01
        # Hold the position for 3 bars before liquidating (in this case, 3 days).
        ctx.hold_bars = 3

That is a lot to unpack! To fully understand how the above works, we need to step through **PyBroker's** execution model for running backtests.

When the backtest runs, the ```buy_low``` function will be called separately for ```AAPL``` and ```MSFT``` on every bar of data. Here, each bar corresponds to a single day of data for the ticker that was retrieved from Yahoo Finance.

An [ExecContext](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext) (```ctx```) will be passed to ```buy_low```. The ```ExecContext``` will contain data for the current ticker symbol being executed using the trading rules in ```buy_low```. For instance, [ctx.close](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.close) will contain all of the close prices up until the most recent bar of the current ticker symbol (```AAPL``` or ```MSFT```). The latest close price is retrieved with ```ctx.close[-1]```.

The ```ExecContext``` is also used to place a buy order. The number of shares to purchase is set in ```buy_low``` using [ctx.buy_shares](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.buy_shares), and the number of shares is calculated with [ctx.calc_target_shares](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.calc_target_shares). In this case, the number of shares to buy will be equal to 25% of the portfolio.

The limit price of the order is set by [buy_limit_price](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.buy_limit_price). If satisifed, the buy order will be filled on the next bar. The time at which the order is filled can be configured with [StrategyConfig.buy_delay](https://www.pybroker.com/en/latest/reference/pybroker.config.html#pybroker.config.StrategyConfig.buy_delay), and its fill price can be set with [ExecContext.buy_fill_price](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.buy_fill_price). By default, buy orders are filled on the next bar (```buy_delay=1```) and at a [fill price equal to the midpoint between that bar's open and close price](https://www.pybroker.com/en/latest/reference/pybroker.common.html#pybroker.common.PriceType.MIDDLE).

Finally, [ctx.hold_bars](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.hold_bars) specifies how many bars to hold the position for before liquidating it. When liquidated, the shares are sold at market price equal to [ExecContext.sell_fill_price](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.sell_fill_price), which is configurable and also defaults to the midpoint between the bar's open and close price.

The ```ExecContext``` contains many other fields containing information about current positions, orders, OHLC prices and more. [You can read more on the reference documentation of ExecContext](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext).

The ```buy_low``` rules are then added to the ```Strategy``` for ```AAPL``` and ```MSFT``` using [add_execution](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy.add_execution):

In [4]:
strategy.add_execution(buy_low, ['AAPL', 'MSFT'])

## Adding a Second Execution

We are not limited to only using one set of trading rules for a single group of tickers. Another execution that implements different trading rules for different tickers can be added to the same ```Strategy```.

Let's implement another set of trading rules that are similar as before, but are for a short strategy:

In [5]:
def short_high(ctx):
    # If shares were already shorted and that position is currently being held, then return.
    if ctx.short_pos():
        return
    # If the latest close price is more than the previous day's high price,
    # then place a sell order.
    if len(ctx.high) >= 2 and ctx.close[-1] > ctx.high[-2]:
        # Short 100 shares.
        ctx.sell_shares = 100
        # Cover the shares after 2 bars (in this case, 2 days).
        ctx.hold_bars = 2

And let's trade these rules on ```TSLA```:

In [6]:
strategy.add_execution(short_high, ['TSLA'])

## Running a Backtest

Running a backtest can be done by calling the [backtest](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy.backtest) method on the ```Strategy```:

In [7]:
result = strategy.backtest(calc_bootstrap=False)

Backtesting: 2017-03-01 00:00:00 to 2022-03-01 00:00:00

Loading bar data...
[*********************100%***********************]  3 of 3 completed
Loaded bar data: 0:00:02 

Test split: 2017-03-01 05:00:00 to 2022-02-28 05:00:00


100% (1259 of 1259) |####################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:05


That was fast! Notice that an argument ```calc_bootstrap=False``` was passed in to disable calculating bootstrap metrics. We can ignore that for now until the next notebook about bootstrap metrics.

The ```result``` returned is an instance of [TestResult](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.TestResult). Let's take a look at the results, starting with the daily balances of our portfolio:

In [8]:
result.portfolio

,cash,equity,margin,market_value,pnl,unrealized_pnl,fees
date,,,,,,,
2017-03-01 05:00:00,500000.00,500000.00,0.00,500000.00,0.00,0.00,0.0
2017-03-02 05:00:00,500000.00,500000.00,0.00,500000.00,0.00,0.00,0.0
2017-03-03 05:00:00,375169.60,500585.60,0.00,500585.60,0.00,585.60,0.0
2017-03-06 05:00:00,375169.60,500624.63,0.00,500624.63,0.00,624.63,0.0
2017-03-07 05:00:00,375169.60,500878.40,0.00,500878.40,0.00,878.40,0.0
...,...,...,...,...,...,...,...
2022-02-22 05:00:00,327443.87,675653.40,0.00,675653.40,181339.03,-5685.63,0.0
2022-02-23 05:00:00,327443.87,666642.14,0.00,666642.14,181339.03,-14696.89,0.0
2022-02-24 05:00:00,665740.20,665740.20,0.00,665740.20,165740.20,0.00,0.0


Likewise, the result contains the daily balance of each position that was held:

In [9]:
result.positions

long_shares  short_shares   close     equity  \
symbol date                                                                
MSFT   2017-03-03 05:00:00         1952             0   64.25  125416.00   
       2017-03-06 05:00:00         1952             0   64.27  125455.03   
       2017-03-07 05:00:00         1952             0   64.40  125708.80   
       2017-03-14 04:00:00         1937             0   64.41  124762.18   
       2017-03-15 04:00:00         1937             0   64.75  125420.75   
...                                 ...           ...     ...        ...   
       2022-02-22 05:00:00          610             0  287.72  175509.20   
AAPL   2022-02-22 05:00:00         1051             0  164.32  172700.33   
MSFT   2022-02-23 05:00:00          610             0  280.27  170964.69   
AAPL   2022-02-23 05:00:00         1051             0  160.07  168233.58   
TSLA   2022-02-28 05:00:00            0           100  290.14       0.00   

                            market_value    margin  unrealized_pnl  
symbol date                                                         
MSFT   2017-03-03 05:00:00     125416.00      0.00          585.60  
       2017-03-06 05:00:00     125455.03      0.00          624.63  
       2017-03-07 05:00:00     125708.80      0.00          878.40  
       2017-03-14 04:00:00     124762.18      0.00          116.23  
       2017-03-15 04:00:00     125420.75      0.00          774.80  
...                                  ...       ...             ...  
       2022-02-22 05:00:00     175509.20      0.00        -1439.60  
AAPL   2022-02-22 05:00:00     172700.33      0.00        -4246.03  
MSFT   2022-02-23 05:00:00     170964.69      0.00        -5984.11  
AAPL   2022-02-23 05:00:00     168233.58      0.00        -8712.78  
TSLA   2022-02-28 05:00:00       -821.33  29014.33         -821.33  

[938 rows x 7 columns]

The trades that were made for each entry and exit:

In [10]:
result.trades

,type,symbol,entry_date,exit_date,entry,exit,shares,pnl,return_pct,agg_pnl,bars,pnl_per_bar
id,,,,,,,,,,,,
1,long,MSFT,2017-03-03 05:00:00,2017-03-08 05:00:00,63.95,64.67,1952,1405.44,1.13,1405.44,3,468.48
2,long,MSFT,2017-03-14 04:00:00,2017-03-17 04:00:00,64.35,64.96,1937,1181.57,0.95,2587.01,3,393.86
3,short,TSLA,2017-03-15 04:00:00,2017-03-17 04:00:00,17.18,17.55,100,-37.00,-2.11,2550.01,2,-18.50
4,short,TSLA,2017-03-27 04:00:00,2017-03-29 04:00:00,17.68,18.50,100,-82.00,-4.43,2468.01,2,-41.00
5,short,TSLA,2017-04-04 04:00:00,2017-04-06 04:00:00,19.98,19.87,100,11.00,0.55,2479.01,2,5.50
...,...,...,...,...,...,...,...,...,...,...,...,...
384,long,AAPL,2022-02-11 05:00:00,2022-02-16 05:00:00,170.56,171.69,1029,1162.77,0.66,180785.17,3,387.59
385,long,MSFT,2022-02-11 05:00:00,2022-02-16 05:00:00,299.26,297.27,586,-1166.14,-0.66,179619.03,3,-388.71
386,short,TSLA,2022-02-16 05:00:00,2022-02-18 05:00:00,304.61,287.41,100,1720.00,5.98,181339.03,2,860.00


And all of the orders that were placed:

In [11]:
result.orders

,type,symbol,date,shares,limit_price,fill_price,fees
id,,,,,,,
1,buy,MSFT,2017-03-03 05:00:00,1952,64.00,63.95,0.0
2,sell,MSFT,2017-03-08 05:00:00,1952,NaN,64.67,0.0
3,buy,MSFT,2017-03-14 04:00:00,1937,64.70,64.35,0.0
4,sell,TSLA,2017-03-15 04:00:00,100,NaN,17.18,0.0
5,sell,MSFT,2017-03-17 04:00:00,1937,NaN,64.96,0.0
...,...,...,...,...,...,...,...
773,buy,AAPL,2022-02-18 05:00:00,1051,168.87,168.36,0.0
774,buy,MSFT,2022-02-18 05:00:00,610,290.72,290.08,0.0
775,sell,AAPL,2022-02-24 05:00:00,1051,NaN,157.43,0.0


The result also contains a DataFrame of metrics calculated from the returns of the backtest:

In [12]:
result.metrics_df

,name,value
0,trade_count,388.000000
1,initial_market_value,500000.000000
2,end_market_value,693111.870000
3,total_pnl,165740.200000
4,total_return_pct,33.148040
5,total_profit,403511.080000
6,total_loss,-237770.880000
7,total_fees,0.000000
8,max_drawdown,-56721.600000
9,max_drawdown_pct,-7.908429


[You can read about what these metrics mean here](https://www.pybroker.com/en/latest/reference/pybroker.eval.html#pybroker.eval.EvalMetrics).

## Filtering Backtest Data

Lastly, it is possible to filter the data used for the backtest. For example, the data can be filtered to only contain bars for Mondays. In effect, this limits the strategy to only trade on Mondays:

In [13]:
result = strategy.backtest(days='mon', train_size=0.5, calc_bootstrap=False)
result.orders

Backtesting: 2017-03-01 00:00:00 to 2022-03-01 00:00:00

Loaded cached bar data.

Test split: 2019-09-09 04:00:00 to 2022-02-28 05:00:00


100% (119 of 119) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:00


,type,symbol,date,shares,limit_price,fill_price,fees
id,,,,,,,
1,sell,TSLA,2019-09-23 04:00:00,100,NaN,16.15,0.0
2,buy,TSLA,2019-10-07 04:00:00,100,NaN,15.57,0.0
3,sell,TSLA,2019-10-21 04:00:00,100,NaN,16.99,0.0
4,buy,TSLA,2019-11-04 05:00:00,100,NaN,21.04,0.0
5,sell,TSLA,2019-11-18 05:00:00,100,NaN,23.31,0.0
...,...,...,...,...,...,...,...
92,sell,TSLA,2022-02-07 05:00:00,100,NaN,308.41,0.0
93,sell,AAPL,2022-02-14 05:00:00,791,NaN,168.07,0.0
94,buy,MSFT,2022-02-14 05:00:00,459,300.94,294.06,0.0


Note that the data did not need to be downloaded again from Yahoo Finance, since ```DataSource``` caching was enabled and because the cached data only needed to be filtered.

Additionally, the data can be filtered by time range (e.g. 9:30-10:30AM) using the [between_time](https://www.pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy.backtest) argument.

From the metrics earlier, it seems like we already have a profitable strategy. But not so fast! We may be getting fooled by randomness. [Next, we will learn about using bootstrapping to further evaluate our trading strategies](https://www.pybroker.com/en/latest/notebooks/3.%20Evaluate%20using%20Bootstrap%20Metrics.html).